In [ ]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

If we want to have **reproducible results**, the environment variable `PYTHONHASHSEED` has to be set to a fixed value, for example to `0`.
Below we check that this environment is set so that results are reproducible.
In order to set this variable we have to use the following sequence of commands in the anaconda shell.  
```
conda activate ai
conda env config vars set PYTHONHASHSEED=0
conda activate ai
```
It is necessary to reactivate the environment `ai` for the setting to take effect.

In [ ]:
import os
os.environ['PYTHONHASHSEED']

In [ ]:
%load_ext nb_mypy

# A Backtracking Solver for CSPs

In [ ]:
from typing import TypeVar

In [ ]:
Value      = TypeVar('Value')
Element    = TypeVar('Element')
Variable   = str
Formula    = str
CSP        = tuple[set[Variable] | list[Variable], set[Value], set[Formula]]
ACSP       = tuple[set[Variable] | list[Variable], set[Value], list[tuple[Formula, set[Variable]]]]
Assignment = dict[Variable, Value]

## Utility Functions

The module `ast` provides a parser for Python expressions.  This is needed as we need to compute the set of variables that occur in a given constraint.

In [ ]:
import ast

The function `collect_variables(expr)` takes a string `expr` that can be interpreted as a Python expression as input and collects all variables occurring in `expr`.  It takes care to remove the function symbols from the names returned by `collect_variables`.

In [ ]:
def collect_variables(expression: Formula) -> set[Variable]: 
    tree = ast.parse(expression)
    return { node.id for node in ast.walk(tree) 
                     if  isinstance(node, ast.Name) 
                     if  node.id not in dir(__builtins__)
           }

Below, `dir(__builtins__)` returns a list containing all predefined variables, functions, and classes. 

In [ ]:
dir(__builtins__)

In [ ]:
collect_variables('abs(x - y) + abs(z1 - z2)')

The function `arb(S)` takes a set `S` as input and returns an arbitrary element from 
this set.

In [ ]:
def arb(S: set[Element]) -> Element:
    for x in S:
        return x
    return None # type: ignore

## The Backtracking Solver

The following forward declarations are needed by the type checker.

In [ ]:
def backtrack_search(Assgnmnt:     Assignment,
                     P:            ACSP, 
                     debug:        bool, 
                     animate:      bool, 
                     problem_size: int | None) -> Assignment | None:
    return None

In [ ]:
def is_consistent(var:         Variable, 
                  value:       Value, 
                  Assgnmnt:    Assignment, 
                  Constraints: list[tuple[Formula, set[Variable]]]) -> bool:
    return None # type: ignore

The procedure `solve(P)` takes a *constraint satisfaction problem* 
`P` as input.  Here `P` is a triple of the form 
$$ \mathcal{P} = \langle \mathtt{Variables}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
where 
- `Variables` is a set of strings which serve as *variables*,
- `Values` is a set of *values* that can be assigned 
  to the variables in the set `Variables`.
- `Constraints` is a set of formulas from first order logic.  
  
The second argument `debug` controlls the printing of debugging information.  If set to true, every partial assignment that is created during the search, will be printed.
  
The main purpose of the function `solve` is to convert the CSP `P` into an 
*augmented CSP* where every constraint $f$ is annotated with
the variables ocurring in $f$.  This annotated CSP is then solved using the function
`backtrack_search`.

In [ ]:
def solve(P:            CSP, 
          debug:        bool=False, 
          animate:      bool=False, 
          problem_size: int|None=None) -> Assignment | None:
    Variables, Values, Constraints = P
    csp = (Variables, Values, [(f, collect_variables(f)) for f in Constraints])
    return backtrack_search({}, csp, debug, animate, problem_size)

The function `backtrack_search` takes two arguments:
- `Assignment` is a partial variable assignment that is represented as a dictionary.  Initially, this assignment will be the empty dictionary.  Every recursive call of `backtrack_search` adds the assignment of one variable to the given assignment.  The important invariant of recursive calls of `backtrack_search` is that `Assignment` is *consistent*, i.e. all constraints $f$ that contain only variables from the set $\mathtt{dom}(\mathtt{Assignment})$ are satisfied.
- `P` is an *augmented constraint satisfaction problem*, 
   i.e. `P` is a triple of the form 
   $$ \mathcal{P} = \langle \mathtt{Vars}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
    where 
    - $\mathtt{Vars}$ is a set of strings which serve as *variables*,
    - $\mathtt{Values}$ is a set of *values* that can be assigned 
      to the variables in $\mathtt{Vars}$.
    - $\mathtt{Constraints}$ is a set of pairs of the form $(f, V)$ where $f$ is a Boolean Python 
      expression, while $V$ is the set of variables occuring in $f$.
- `debug` is a Boolean flag.  If this flag is true, every partial assignment is printed.      
The function `backtrack_search` tries to find a solution of `P` by recursively augmenting `Assignment`.

Normally, the `Variables` stored in the CSP are represented as a set.  However, for didactical purposes it is also possible to store the variables as a list.  This way, the order in which variables are chosen can be controlled.

In [ ]:
def backtrack_search(Assgnmnt:     Assignment,
                     P:            ACSP, 
                     debug:        bool, 
                     animate:      bool, 
                     problem_size: int | None) -> Assignment | None:
    if debug and not animate:
        print(Assgnmnt)
    if animate:
        if problem_size == None:
            display(show_solution(Assgnmnt, width="50%")) # type: ignore
        else: 
            display(show_solution(Assgnmnt, problem_size, width="50%")) # type: ignore
    Variables, Values, Constraints = P
    if len(Assgnmnt) == len(Variables):
        return Assgnmnt
    if isinstance(Variables, set):
        var = arb(Variables - Assgnmnt.keys())
    else: # if Variables is a list we choose the first unassigned variable
        var = [x for x in Variables if x not in Assgnmnt][0]
    for value in Values:
        if is_consistent(var, value, Assgnmnt, Constraints):
            NewAss = Assgnmnt.copy()
            NewAss[var] = value
            Solution = backtrack_search(NewAss, P, debug, animate, problem_size)
            if Solution != None:
                return Solution
    return None

The function $\texttt{is_consistent}(\texttt{var}, \texttt{value}, \texttt{Assignment}, \texttt{Constraints})$ takes four arguments:
- `var` is a variable that does not occur in $\texttt{Assignment}$,
- `value` is a value that can be substituted for this variable,
- `Assignment` is a *consistent* partial variable assignment. 
- `Constraints` is a set of pairs of the form $\langle f, V \rangle$ where $f$ is a formula and $V$ is the set of variables occurring in $f$.

This function returns `True` iff the partial variable assignment 
$$\texttt{Assignment} \cup \bigl\{\langle\texttt{var} \mapsto\texttt{value}\rangle\bigr\}$$
is consistent with all the constraints $f$ occurring in `Constraints`.

**Note** that the function `eval` mutates the given variable assignment.  Therefore it is necessary to copy the given `Assignment`.  

In [ ]:
def is_consistent(var:         Variable, 
                  value:       Value, 
                  Assgnmnt:    Assignment, 
                  Constraints: list[tuple[Formula, set[Variable]]]) -> bool:
    NewAssign      = Assgnmnt.copy()
    NewAssign[var] = value
    return all(eval(f, NewAssign) for (f, Vs) in Constraints
                                  if  var in Vs and Vs <= NewAssign.keys()
              )

# Map Coloring

In [ ]:
%%capture 
%run 01-MapColoring.ipynb

In [ ]:
%unload_ext nb_mypy

In [ ]:
P = map_coloring_csp()
P

In [ ]:
%%time
Solution = solve(P, animate=True)

In [ ]:
show_solution(Solution)

If we reorder the variables appropriately, the solver never needs to backtrack.

In [ ]:
P2 = (['SA', 'WA',  'NT', 'Q', 'NSW', 'V', 'T'], P[1], P[2])

In [ ]:
%%time
Solution = solve(P2, animate=True)

## Solving the Eight-Queens-Puzzle

In [ ]:
%%capture
%run 02-NQueensProblemCSP.ipynb

In [ ]:
P = create_csp(7)

For the 7-queens puzzle we only have to backtrack once, as the animation below shows.

In [ ]:
%%time
Solution = solve(P, animate=True, problem_size=7)
print(f'Solution = {Solution}')

Backtracking is able to solve the $16$ queens problem in about $13$ seconds.

In [ ]:
P = create_csp(16)

In [ ]:
%%time
Solution = solve(P)
print(f'Solution = {Solution}')

In [ ]:
show_solution(Solution, 16)

There are $16^{16} = 18\,446\,744\,073\,709\,551\,616$ possible assignments for the 16 queens problem.

In [ ]:
16**16

## Solving the *Zebra Puzzle*

In [ ]:
%%capture
%run 05-Zebra.ipynb

In [ ]:
zebra = zebra_csp()

Backtracking takes less than a minute to solve the [Zebra Puzzle](https://en.wikipedia.org/wiki/Zebra_Puzzle).
We develop a better algorithm soon.

In [ ]:
%%time
Solution = solve(zebra)

In [ ]:
show_solution(Solution)

## Solving the Cypto-Arithmetic Puzzle

In [ ]:
%%capture
%run 06-CryptoArithmetic.ipynb

In [ ]:
csp = crypto_csp()
csp

It takes about a minute to solve the crypto-arithmetic puzzle.

In [ ]:
%%time
Solution = solve(csp)

In [ ]:
show_solution(Solution)

Let's try the harder version.

In [ ]:
csp = crypto_csp_hard()

In [ ]:
%%time
Solution = solve(csp)

It takes about 3 minutes and 30 seconds to solve the hard version.